In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread
from csbdeep.utils.tf import keras_import
keras = keras_import()
from stardist import random_label_cmap
import os
from glob import glob

np.random.seed(0)
cmap = random_label_cmap()

In [2]:
pth = r'C:\Users\Donald\Desktop\AD_IF_Images\AD brain tissue images_ImmunoFluorescence\20231115_Shared\5xFAD 12MO F3-1_IF slide1\combined_ims'  # path of tif files
WSIs = sorted(glob(f'{pth}\*.tif'))

In [3]:
def show_image(img, crop_x, crop_y, tile_size, **kwargs):
    fig, ax = plt.subplots(1, 2, figsize=(16, 8))
    ax[0].imshow(img, **kwargs)
    cropped_img = img[crop_y:crop_y+tile_size, crop_x:crop_x+tile_size]
    ax[1].imshow(cropped_img, **kwargs)
    ax[0].axis('off')
    ax[1].axis('off')
    plt.tight_layout()
    plt.show()

In [4]:
from stardist.models import StarDist2D

# prints a list of available models
StarDist2D.from_pretrained()

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [6]:
# segment DAPI_Cy5 combined im
img = imread(WSIs[0])
img = img/255

labels_Cy5, polys_Cy5 = model.predict_instances_big(img, axes='YX', block_size=4096, min_overlap=128, context=128, n_tiles=(4,4))

effective: block_size=(4096, 4096), min_overlap=(128, 128), context=(128, 128)


100%|██████████| 24/24 [00:46<00:00,  1.96s/it]


In [7]:
# segment DAPI_IBA combined im
img = imread(WSIs[1])
img = img/255

labels_IBA, polys_IBA = model.predict_instances_big(img, axes='YX', block_size=4096, min_overlap=128, context=128, n_tiles=(4,4))

effective: block_size=(4096, 4096), min_overlap=(128, 128), context=(128, 128)


100%|██████████| 24/24 [00:49<00:00,  2.06s/it]


In [5]:
WSIs

['C:\\Users\\Donald\\Desktop\\AD_IF_Images\\AD brain tissue images_ImmunoFluorescence\\20231115_Shared\\5xFAD 12MO F3-1_IF slide1\\combined_ims\\DAPI_Cy5.tif',
 'C:\\Users\\Donald\\Desktop\\AD_IF_Images\\AD brain tissue images_ImmunoFluorescence\\20231115_Shared\\5xFAD 12MO F3-1_IF slide1\\combined_ims\\DAPI_IBA.tif']

In [8]:
from scipy.io import loadmat
import h5py

def load_var_from_mat(mat_file_name, varname):
    try:
        data = loadmat(mat_file_name)
        var = data[varname]
    except:
        data = h5py.File(mat_file_name, 'r')
        var = data[varname][()]
    return var

In [9]:
pth = r'C:\Users\Donald\Desktop\AD_IF_Images\AD brain tissue images_ImmunoFluorescence\20231115_Shared\5xFAD 12MO F3-1_IF slide1\combined_ims\coords_for_stardist'

Cy5_coords = load_var_from_mat(os.path.join(pth, 'Cy5_coords.mat'), "Cy5_coords")

IBA_coords = load_var_from_mat(os.path.join(pth, 'IBA_coords.mat'), "IBA_coords")

In [16]:
adj_x_Cy5 = [x for x in Cy5_coords[:,0]]
adj_y_Cy5 = [y for y in Cy5_coords[:,1]]
adj_Cy5_coords = np.transpose(np.array([adj_x_Cy5, adj_y_Cy5]))

adj_x_IBA = [x for x in IBA_coords[:,0]]
adj_y_IBA = [y for y in IBA_coords[:,1]]
adj_IBA_coords = np.transpose(np.array([adj_x_IBA, adj_y_IBA]))

In [17]:
def is_point_encompassed(xy_point, set_of_points):
    x, y = xy_point
    min_x, max_x = min(point[0] for point in set_of_points), max(point[0] for point in set_of_points)
    min_y, max_y = min(point[1] for point in set_of_points), max(point[1] for point in set_of_points)
    return min_x <= x <= max_x and min_y <= y <= max_y

In [18]:
# for Cy5 coords:

in_tile_inds = []
poly_inds = []

in_tile = []
for num, i in enumerate(polys_Cy5['coord']):
    xy = np.transpose(i)
    # get centroids of poly
    x_cent = np.mean(xy[1])  # reversed order, idk why
    y_cent = np.mean(xy[0])
    
    # fix orientation of coordinates in matrix
    x = [x for x in xy[:,0]]
    y = [y for y in xy[:,1]]
    xy = np.transpose(np.array([x, y]))[:, ::-1]

    
    in_tile.append([xy])  # just for testing, delete later probably
    for xy_cy5 in adj_Cy5_coords:
        
        if is_point_encompassed(xy_cy5, xy):
            in_tile_ind = len(in_tile) - 1
            in_tile_inds.append(in_tile_ind)
            
            poly_inds.append(num)
            print(num, x_cent, y_cent)

0 3772.912109375 3772.3267822265625
17 3772.9813232421875 3772.173583984375
54 4477.9288330078125 4476.66650390625
57 4094.686767578125 4094.0263061523438
59 4903.295166015625 4902.521240234375
61 4075.304443359375 4074.2969970703125
62 3976.2813110351562 3975.68408203125
64 4806.51220703125 4805.0294189453125
65 4272.179260253906 4271.1339111328125
67 4207.4598388671875 4206.7015380859375
68 4352.6402587890625 4351.695556640625
69 4505.1885986328125 4504.2642822265625
72 3824.4797973632812 3823.4819946289062
73 4575.1524658203125 4574.6043701171875
74 4144.0286865234375 4142.88916015625
74 4144.0286865234375 4142.88916015625
75 4097.178955078125 4096.7247314453125
76 4233.422790527344 4232.8948974609375
80 4598.877685546875 4598.4075927734375
81 4311.4617919921875 4310.2733154296875
81 4311.4617919921875 4310.2733154296875
82 4053.21875 4052.6602172851562
83 4626.7076416015625 4626.345947265625
83 4626.7076416015625 4626.345947265625
90 4898.115478515625 4896.8929443359375
90 4898.115

KeyboardInterrupt: 

In [19]:
len(adj_Cy5_coords)

6862

In [20]:
len(polys_Cy5['coord'])

47109